In [1]:
import pywt
import os
import pandas as pd
import librosa
import librosa.display
import glob 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions
from matplotlib.ticker import FormatStrFormatter

from sklearn.preprocessing import StandardScaler
import sklearn

from utils import plot_projections

from scipy.stats import kurtosis, skew
import librosa.display as ld


%matplotlib widget

In [4]:
laser_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/'
laser_files = list(glob.glob(os.path.join(laser_dir, '*.wav')))
laser_files.sort()
# print(laser_files)
print(len(laser_files))

50


In [5]:
orig_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/Original/'
orig_files = list(glob.glob(os.path.join(orig_dir, '*.wav')))
orig_files.sort()
# print(orig_files)
print(len(orig_files))

50


In [6]:
sample_id = 13

In [7]:
# time domain
laser_sample = laser_files[sample_id]
gen_sample = orig_files[sample_id]

laser_audio, sr = librosa.load(laser_sample, res_type='kaiser_fast')
laser_audio = librosa.util.normalize(laser_audio)
print(sr)
gen_audio, sr = librosa.load(gen_sample, res_type='kaiser_fast')
gen_audio = librosa.util.normalize(gen_audio)
print(sr)

22050
22050


In [8]:
fig, ax = plt.subplots(2,1, figsize=(8,10))

ax[0].set(title='Wave Form of Laser Audio')
ld.waveplot(laser_audio, sr=sr, ax=ax[0])
# ax[0].set_xlim([0, 4])
# ax[0].set_ylim([-1, 1])
# ax[0].set_ylim([-0.03, 0.03])
ax[0].grid()

ax[1].set(title='Wave Form of Original Audio')
ld.waveplot(gen_audio, sr=sr, ax=ax[1])
# ax[1].set_xlim([0, 4])
# ax[1].set_ylim([-1, 1])
# ax[1].set_ylim([-0.03, 0.03])
ax[1].grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Aligning the two audios


In [9]:
n_fft = 44100
hop_size = 22050

laser_chroma = librosa.feature.chroma_stft(y=laser_audio, sr=sr, tuning=0, norm=2,
                                         hop_length=hop_size, n_fft=n_fft)
original_chroma = librosa.feature.chroma_stft(y=gen_audio, sr=sr, tuning=0, norm=2,
                                         hop_length=hop_size, n_fft=n_fft)

plt.figure(figsize=(16, 8))
plt.subplot(2, 1, 1)
plt.title('Chroma Representation of $X_1$')
librosa.display.specshow(laser_chroma, x_axis='time',
                         y_axis='chroma', cmap='gray_r', hop_length=hop_size)
plt.colorbar()
plt.subplot(2, 1, 2)
plt.title('Chroma Representation of $X_2$')
librosa.display.specshow(original_chroma, x_axis='time',
                         y_axis='chroma', cmap='gray_r', hop_length=hop_size)
plt.colorbar()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
D, wp = librosa.sequence.dtw(X=laser_chroma, Y=original_chroma, metric='cosine')
wp_s = np.asarray(wp) * hop_size / sr

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
librosa.display.specshow(D, x_axis='time', y_axis='time',
                         cmap='gray_r', hop_length=hop_size)
imax = ax.imshow(D, cmap=plt.get_cmap('gray_r'),
                 origin='lower', interpolation='nearest', aspect='auto')
ax.plot(wp_s[:, 1], wp_s[:, 0], marker='o', color='r')
plt.title('Warping Path on Acc. Cost Matrix $D$')
plt.colorbar

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<function matplotlib.pyplot.colorbar(mappable=None, cax=None, ax=None, **kw)>

In [11]:
fig, ax = plt.subplots(2,1, figsize=(8,10))

ax[0].set(title='Wave Form of Laser Audio')
ld.waveplot(laser_audio, sr=sr, ax=ax[0])
ax[0].set_xlim([1, 1.2])
ax[0].set_ylim([-1, 1])
ax[0].grid()

ax[1].set(title='Wave Form of Original Audio')
ld.waveplot(gen_audio, sr=sr, ax=ax[1])
ax[1].set_xlim([1, 1.2])
ax[1].set_ylim([-1, 1])
ax[1].grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
stft_laser = librosa.stft(laser_audio, n_fft=256)
S_db_laser = librosa.amplitude_to_db(np.abs(stft_laser), ref=np.max)

stft_gen = librosa.stft(gen_audio, n_fft=256)
S_db_gen = librosa.amplitude_to_db(np.abs(stft_gen), ref=np.max)

In [13]:
fig, axes = plt.subplots(2, 1, figsize=(9,6))
# ax = axes.flatten()

img_laser = librosa.display.specshow(S_db_laser, x_axis='time', y_axis='log', sr=sr, ax=axes[0])
axes[0].set(title='Laser Audio Spectogram')
axes[0].label_outer()


img_gen = librosa.display.specshow(S_db_gen, x_axis='time', y_axis='log', sr=sr, ax=axes[1])
axes[1].set(title='Genuine Audio Spectogram')
axes[1].label_outer()

fig.colorbar(img_laser, ax=axes, format="%+2.f dB")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Zoomed In

In [15]:
fig, axes = plt.subplots(2, 1, figsize=(9,6))
# ax = axes.flatten()

img_laser = librosa.display.specshow(S_db_laser, x_axis='time', y_axis='log', sr=sr, ax=axes[0])
axes[0].set(title='Laser Audio Spectogram')
axes[0].label_outer()
axes[0].set_xlim([5, 13])
# ax[0].set_ylim([-1, 1])
# ax[0].grid()

img_gen = librosa.display.specshow(S_db_gen, x_axis='time', y_axis='log', sr=sr, ax=axes[1])
axes[1].set(title='Genuine Audio Spectogram')
axes[1].label_outer()
axes[1].set_xlim([5, 13])
# ax[0].set_ylim([-1, 1])
# ax[0].grid()

fig.colorbar(img_laser, ax=axes, format="%+2.f dB")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Distrubtion of Low Frequency Regions

In [13]:
S_db_laser.shape
S_db_laser[:,0].shape

(129,)

### Coeff 0

In [14]:
fig, ax = plt.subplots(2, 1, figsize=(9,6))

ax[0].hist(S_db_laser[:,0], bins='auto')
ax[0].set(title='Laser Audio Distribution for Coeff 0')
ax[0].label_outer()

ax[1].hist(S_db_gen[:,0], bins='auto')
ax[0].set(title='Genuine Audio Distribution for Coeff 0')
ax[0].label_outer()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Coeff 1

In [15]:
fig, ax = plt.subplots(2, 1, figsize=(9,6))

ax[0].hist(S_db_laser[:,1], bins='auto')
ax[0].set(title='Laser Audio Distribution for Coeff 1')
ax[0].label_outer()

ax[1].hist(S_db_gen[:,1], bins='auto')
ax[0].set(title='Genuine Audio Distribution for Coeff 1')
ax[0].label_outer()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Coeff 2


In [16]:
fig, ax = plt.subplots(2, 1, figsize=(9,6))

ax[0].hist(S_db_laser[:,2], bins='auto')
ax[0].set(title='Laser Audio Distribution for Coeff 2')
ax[0].label_outer()

ax[1].hist(S_db_gen[:,2], bins='auto')
ax[0].set(title='Genuine Audio Distribution for Coeff 2')
ax[0].label_outer()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Coeff 3

In [17]:
fig, ax = plt.subplots(2, 1, figsize=(9,6))

ax[0].hist(S_db_laser[:,3], bins='auto')
ax[0].set(title='Laser Audio Distribution for Coeff 3')
ax[0].label_outer()

ax[1].hist(S_db_gen[:,3], bins='auto')
ax[0].set(title='Genuine Audio Distribution for Coeff 3')
ax[0].label_outer()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Coeff 4

In [18]:
fig, ax = plt.subplots(2, 1, figsize=(9,6))

ax[0].hist(S_db_laser[:,4], bins='auto')
ax[0].set(title='Laser Audio Distribution for Coeff 3')
ax[0].label_outer()

ax[1].hist(S_db_gen[:,4], bins='auto')
ax[0].set(title='Genuine Audio Distribution for Coeff 3')
ax[0].label_outer()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …